In [1]:
import numpy as np

First import the theoretical prediction for inclusive Vcb that you want to use. Choices are: <br>
a) Normalized or centralized moments <br>
b) Charm mass in the MS-bar or the kinetic mass scheme

In this notebook we demonstrate the moments in the MS-bar scheme.

In [2]:
from InclusiveVcb import NormalizedMomentsMS, CentralizedMomentsMS

Create instances of the theoretical prediction.

In [3]:
norm = NormalizedMomentsMS()
cent = CentralizedMomentsMS()

Each instance contains the same methods and takes the same parameters as input - they are contained in .data of the instance. For reference see the doc.

In [4]:
print(norm.__doc__)

Total Rate and first four normalized q2-moments in the kinetic scheme
    for the bottom quark and the MS-bar scheme for the charm quark mass.

    Attributes
    ----------
    mus : float
        Scale for the strong coupling a_s (default is 4.546)
    mu0 : float
        Scale for the charm quark mass (default is 2)
    mcMS : float
        Mass of the charm quark in the MS-bar scheme (default is 1.093 for mu0=2)
    mbkin : float
        Mass of the bottom quark in the kinetic scheme (default is 4.546)

    Methods
    -------
    data : None
        Holds all parameters for inclusive Vcb - see its own doc.
    total_rate : (Vcb, mbkin, mcMS, muG, sB, rE, sqB, sE, rG, rhoD, mupi)
        Total rate up to a_s^3 and 1/mb^4
    q2_moment_i : (q_cut, mbkin, mcMS, muG, sB, rE, sqB, sE, rG, rhoD, mupi)
        The i-th q2-moment up to a_s and 1/mb^4. First argument must be provided as np.array.
    covariance_matrix : (cuts, shifts, multi=1, decorr=None)
        Covariance matrix of the 

If desired you could specify a different scale for the strong coupling. It is automatically run to the scale provided.

In [5]:
print(norm.data.api4)
print(NormalizedMomentsMS(mus=3).data.api4)

0.06952577501550422
0.08057073560277814


You can access all parameters via the method data. We provide a set of default HQE parameters (you could also change these at instatiation, however that is only necessary if you want to use a different default input vector for convenience - see below).

In [6]:
print(norm.data.muG)
print(norm.data.rhoD)

0.36156
0.145


The total rate is called with the parameters as given above. If you want to use a default input for the HQE parameters there is a vector default in data.

In [7]:
norm.total_rate(0.042, 4.546, 1.093, 0.36156, -0.132, 0.019, -0.8, -0.072, -0.006, 0.145, 0.432375)

0.10287512718674202

In [8]:
norm.total_rate(0.042, *norm.data.default)

0.10287512718674202

The same can be done for the first - fourth moment. Note that the first input is not Vcb here, but a numpy array of the cuts of interest. Again see the doc.

In [9]:
print(norm.q2_moment_1.__doc__)


        First normalized q2moment up to a_s and 1/mb^4 for the bottom quark in the kinetic
        and the charm quark in the MS-bar scheme.

        Parameters
        ----------
        q_cut : numpy.array
            Cuts on q2
        mbkin : float
            Mass of the bottom quark in the kinetic scheme
        mcMS : float
            Mass of the charm quark in the MS-bar scheme
        muG : float
            HQE parameter muG
        sB : float
            HQE parameter sB
        rE : float
            HQE parameter rE
        sqB : float
            HQE parameter sqB
        sE : float
            HQE parameter sE
        rG : float
            HQE parameter rG
        rhoD : float
            HQE parameter rhoD
        mupi : float
            HQE parameter mupi

        Return
        ------
        numpy.array
            Array for the first q2moment at given cuts.
        


In [10]:
norm.q2_moment_1(np.array([1,2,3]), *norm.data.default)

array([4.85077151, 5.42853611, 6.01606593])

In [11]:
norm.q2_moment_3(np.array([1,2,3]), *norm.data.default)

array([204.52666927, 239.39988765, 284.53789033])

This can be done in the same manner for the centralized moments.

In [12]:
cent.q2_moment_1(np.array([1,2,3]), *norm.data.default)

array([4.85077151, 5.42853611, 6.01606593])

In [13]:
cent.q2_moment_3(np.array([1,2,3]), *norm.data.default)

array([4.11278855, 2.87871278, 1.85350586])

Note that you can set all perturbative corrections to zero by creating an instance and deleting api4 from it.

In [14]:
nopert = NormalizedMomentsMS()
nopert.data.api4 = 0
nopert.q2_moment_1(np.array([1,2,3]), *nopert.data.default)

array([4.81664565, 5.40001094, 5.99271937])

You can also set all 1/mb contributions to zero.

In [15]:
norm.q2_moment_1(np.array([1,2,3]), *norm.data.noHQE)

array([5.22276315, 5.81068567, 6.41042204])

Finally each instance contains a method to determine the covariance matrix. It depicts the covariance between the moments and the moments of different cuts. It has the form <br> (rate, q2mom1(cut1),...,q2mom4(cut1),..., q2mom1(cutn),...q2mom4(cutn)) ^ 2 <br>
Note that the rate is always fully decorelated from the moments.

In [16]:
print(norm.covariance_matrix.__doc__)


        Covariance matrix between the total rate and the q2moments.

        Parameters
        ----------
        cuts : numpy.array
            Array of the cuts on q2
        shifts : dict
            Keys of the dictionary give as the parameters of the moments
            and values as their shift in percent. Can also shift the scale
            of a_s and the masses.
            Example: {muG:0.2, mus:0.1}
        multi : int
            Enlarges dimension of the covariance matrix if several sets of
            q2moments are used.
            Example: multi=2 yields form (rate, mom1,...,mom4, mom1, ..., mom4)
        decorr : None, 'Block' or 2d-list
            None: no decorrelation scenario
            'Block': Assumes no correlation between different q2 sets.
            list: decreasing decorrelation in percent for
                list[0]: decorrelation between the cuts
                list[1]: decorrelation between the sets of moments

        Return
        ------
        

Provide the cuts you are interested in and the shifts for the parameters that can vary in percent (except if the scale mus changes. That shift must be provided as an absolute like mb/2). Below we vary muG by 20% and mus by mbkin/2.

In [17]:
cuts = np.array([1,2])
shift = {'muG':0.2, 'mus':norm.data.mbkin / 2}
norm.covariance_matrix(cuts, shift)

array([[8.70924151e-07, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 2.17323730e-03, 2.21317609e-03, 3.10976192e-02,
        3.35883729e-02, 3.76705754e-01, 4.18715843e-01, 4.39018749e+00,
        4.95542687e+00],
       [0.00000000e+00, 2.21317609e-03, 2.25750777e-03, 3.17010844e-02,
        3.42763757e-02, 3.84093277e-01, 4.27242689e-01, 4.47459861e+00,
        5.05350157e+00],
       [0.00000000e+00, 3.10976192e-02, 3.17010844e-02, 4.45266090e-01,
        4.81245832e-01, 5.39447455e+00, 5.99882057e+00, 6.28533313e+01,
        7.09701614e+01],
       [0.00000000e+00, 3.35883729e-02, 3.42763757e-02, 4.81245832e-01,
        5.20490906e-01, 5.83114016e+00, 6.48752513e+00, 6.79244572e+01,
        7.67238150e+01],
       [0.00000000e+00, 3.76705754e-01, 3.84093277e-01, 5.39447455e+00,
        5.83114016e+00, 6.53566126e+01, 7.26852152e+01, 7.61462280e+02,
        8.5

You can also choose a decorrelation scenario or have the form of the matrix multiplied for several datasets, as explained in the doc.

In [18]:
norm.covariance_matrix(cuts, shift, decorr=[0.97,0.99])

array([[8.70924151e-07, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 2.17323730e-03, 2.14678080e-03, 2.92597499e-02,
        3.06552011e-02, 3.72938696e-01, 4.02092824e-01, 4.08942013e+00,
        4.47745747e+00],
       [0.00000000e+00, 2.14678080e-03, 2.25750777e-03, 3.07500519e-02,
        3.22506419e-02, 3.50551564e-01, 4.22970262e-01, 4.29695704e+00,
        4.70729123e+00],
       [0.00000000e+00, 2.92597499e-02, 3.07500519e-02, 4.45266090e-01,
        4.66808457e-01, 5.07566110e+00, 5.47496156e+00, 6.22247980e+01,
        6.81526460e+01],
       [0.00000000e+00, 3.06552011e-02, 3.22506419e-02, 4.66808457e-01,
        5.20490906e-01, 5.65620596e+00, 6.10411239e+00, 6.19928181e+01,
        7.59565769e+01],
       [0.00000000e+00, 3.72938696e-01, 3.50551564e-01, 5.07566110e+00,
        5.65620596e+00, 6.53566126e+01, 7.05046587e+01, 7.16459860e+02,
        7.8